In [1]:
%load_ext autoreload
%autoreload 2

# https://scipy-cookbook.readthedocs.io/items/bundle_adjustment.html
import urllib
import bz2
import os
import numpy as np

BASE_URL = "http://grail.cs.washington.edu/projects/bal/data/ladybug/"
FILE_NAME = "problem-49-7776-pre.txt.bz2"
URL = BASE_URL + FILE_NAME

if not os.path.isfile(FILE_NAME):
    urllib.request.urlretrieve(URL, FILE_NAME)

In [2]:
def read_bal_data(file_name):
    with bz2.open(file_name, "rt") as file:
        n_cameras, n_points, n_observations = map(
            int, file.readline().split())

        camera_indices = np.zeros(n_observations, dtype=int)
        points_2d = np.zeros((n_cameras, n_points, 1, 2))

        for i in range(n_observations):
            camera_index, point_index, x, y = file.readline().split()
            points_2d[int(camera_index), int(point_index)] = [float(x), float(y)]

        calib = {}
        camera_params = np.empty(n_cameras * 9)
        for i in range(n_cameras * 9):
            camera_params[i] = float(file.readline())
        camera_params = camera_params.reshape((n_cameras, -1))

        points_3d = np.empty(n_points * 3)
        for i in range(n_points * 3):
            points_3d[i] = float(file.readline())
        points_3d = points_3d.reshape((n_points, 1, -1))

    return camera_params, points_3d, points_2d

In [3]:
camera_params, _, points2d = read_bal_data(FILE_NAME)


In [21]:
'''
cam.rvec = camera_params[0:3]
cam.tvec = camera_params[3:6]
if update_intrinsic:
    cam.fx = camera_params[6]
    cam.fy = camera_params[7]
if update_distort:
    cam.distort  = camera_params[8:13]
'''
import cv2
calib = dict()
for cid, params in enumerate(camera_params):
    calib[cid] = {"R": cv2.Rodrigues(params[0:3])[0], "tvec": params[3:6], "intr": np.array([[params[6], 0, 0], [0, params[6], 0], [0, 0, 1]]), "distort":np.array([params[7], params[8], 0, 0, 0], dtype=float)}

In [22]:
from pyba.CameraNetwork import CameraNetwork
camNet = CameraNetwork(points2d=points2d, calib=calib)

In [23]:
np.mean(camNet.reprojection_error())

174.46979936316617

In [24]:
from pyba.pyba import bundle_adjust
bundle_adjust(camNet, update_distort=False, update_intrinsic=True, max_num_images=1000)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         5.2517e+09                                    5.00e+12    
       1              4         1.4960e+09      3.76e+09       2.01e+02       5.35e+11    
       2              5         2.0929e+08      1.29e+09       9.38e+02       3.85e+10    
       3              7         1.2667e+08      8.26e+07       3.61e+02       2.09e+10    
       4              8         4.5131e+07      8.15e+07       6.03e+02       4.51e+09    
       5             10         2.8721e+07      1.64e+07       2.90e+02       2.53e+09    
       6             12         2.3438e+07      5.28e+06       1.36e+02       1.87e+09    
       7             13         1.5332e+07      8.11e+06       1.63e+02       6.51e+08    
       8             15         1.3559e+07      1.77e+06       1.54e+02       2.80e+08    
       9             16         1.2066e+07      1.49e+06       2.87e+02       1.64e+08    

 active_mask: array([0., 0., 0., ..., 0., 0., 0.])
        cost: 4929928.542634447
         fun: array([ 16.44849176, -10.08994003,   9.07287686, ...,  -6.56597499,
         0.80971913,  -5.62253594])
        grad: array([-9.39086994e+09,  1.49843252e+11,  1.46984253e+10, ...,
       -5.00264975e+05,  6.10371814e+05, -3.11581332e+07])
         jac: <8204x3637 sparse matrix of type '<class 'numpy.float64'>'
	with 131264 stored elements in Compressed Sparse Row format>
     message: 'The maximum number of function evaluations is exceeded.'
        nfev: 100
        njev: 75
  optimality: 149843252422.39404
      status: 0
     success: False
           x: array([ 0.00305115, -0.0054141 , -0.00106268, ..., -0.36301288,
        0.28582571, -0.01773095])

In [25]:
np.mean(camNet.reprojection_error())

1084036.251123258

In [26]:
camNet.triangulate()

array([[[ -0.04597823,   0.48529548,  -0.94861022]],

       [[ -0.60064826, -16.31248157,  -4.22512892]],

       [[  0.26095218, -31.61228772,  -3.43733708]],

       ...,

       [[  1.29200712,   0.03943001,  -4.55609577]],

       [[  0.53202375,   0.05454598,  -3.32926069]],

       [[  0.61051468,   0.05996293,  -3.39874165]]])